In [1]:
# Import libraries
import py_entitymatching as em
import pandas as pd
import py_stringmatching as sm
import os, sys

# Matching two tables typically consists of the following three steps:
** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# 1. Loading the input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [2]:
#dblp_dataset_path = os.sep.join(['DBLP_ACM', 'DBLP_cleaned.csv'])
#acm_dataset_path = os.sep.join(['DBLP_ACM', 'ACM_cleaned.csv'])

datasets_dir = os.getcwd() + os.sep + 'data\\csv\\Abt-Buy'

abt_dataset_path = datasets_dir + os.sep + 'abtProfiles.csv'
buy_dataset_path = datasets_dir + os.sep + 'buyProfiles.csv'

In [3]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(abt_dataset_path, encoding = "ISO-8859-1", key='id')
B = em.read_csv_metadata(buy_dataset_path, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
print('Number of entities in A: ' + str(len(A)))
print('Number of entities in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of entities in A: 1076
Number of entities in B: 1076
Number of tuples in A X B (i.e the cartesian product): 1157776


In [5]:
A

,id,name,description,price
0,0,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,NaN
1,1,Bose Acoustimass 5 Series III Speaker System - AM53BK,Bose Acoustimass 5 Series III Speaker System - AM53BK/ 2 Dual Cube Speakers With Two 2-1/2' Wide...,399.0
2,2,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecting And Reconnecting Cables/ Compact Design/ 4 A/V ...,49.0
3,3,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Changer/ Variable Line Output/ CD-R/RW Playback Capabili...,NaN
4,4,Bose 27028 161 Bookshelf Pair Speakers In White - 161WH,Bose 161 Bookshelf Speakers In White - 161WH/ Articulated Array Speaker Design/ High-Excursion T...,158.0
...,...,...,...,...
1071,1071,Logitech Cordless Desktop Wave Keyboard And Mouse Combo - 920000264,Logitech Cordless Desktop Wave Keyboard And Mouse Combo - 920000264/ Contoured Wave-Shaped Key D...,79.0
1072,1072,Mitsubishi DLP Black TV Stand - MBS73V,Mitsubishi DLP Black TV Stand - MBS73V/ Matching Base For Mitsubishi WD-73735 And WD-73736 DLP H...,549.0
1073,1073,Logitech Digital Precision PC Gaming Headset - 981000040,Logitech Digital Precision PC Gaming Headset - 981000040/ Cushioned Open-Air Design/ USB And Ana...,49.0
1074,1074,Logitech 2.1 Multimedia Silver Speaker System - Z2300,Logitech 2.1 Multimedia Silver Speaker System - Z2300/ THX-Certified/ 200 Watts Of RMS Power (40...,NaN


In [6]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [7]:
# If the tables are large we can downsample the tables like this
# A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
# len(A1), len(B1)

# 2. Blocking to create candidate tuple pairs
Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.

Magellan provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Refer to [api reference] for more details. The user can mix and match these blockers to form a blocking sequence applied to input tables.



In [8]:
ob = em.OverlapBlocker()
OB = ob.block_tables(A, B, 'name', 'name', l_output_attrs=['name', 'description', 'price'],
                   r_output_attrs=['description', 'name', 'price'])

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_df[r_dummy_overlap_attr] = r_df[r_overlap_attr]
C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\blocker\overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying t

In [9]:
OB

,_id,ltable_id,rtable_id,ltable_name,ltable_description,ltable_price,rtable_description,rtable_name,rtable_price
0,0,100,0,NetGear ProSafe 24 Port Smart Switch - FS726TP,NetGear ProSafe 24 Port Smart Switch - FS726TP/ Two Gigabit Ports Plus Easy Browser/ ProSafe Net...,605.0,Linksys EtherFast 8-Port 10/100 Switch (New/Workgroup),Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W,NaN
1,1,106,0,Netgear ProSafe 16 Port 10/100 Desktop Switch - Purple Finish - FS116P,Netgear ProSafe 16 Port 10/100 Desktop Switch - FS116P/ 16 Auto Speed-Sensing 10/100 RJ-45 Ports...,299.0,Linksys EtherFast 8-Port 10/100 Switch (New/Workgroup),Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W,NaN
2,2,134,0,Linksys EtherFast 4124 24-Port Ethernet Switch - EF4124,Linksys EtherFast 4124 24-Port Ethernet Switch - EF4124/ 24 Autosensing 10/100 Full Duplex Auto ...,119.0,Linksys EtherFast 8-Port 10/100 Switch (New/Workgroup),Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W,NaN
3,3,160,0,Netgear Prosafe 16 Port 10/100 Rackmount Switch - Black Finish - JFS516NA,Netgear Prosafe 16 Port 10/100 Rackmount Switch - JFS516NA/ Sixteen Switched Ports Provide Priva...,131.0,Linksys EtherFast 8-Port 10/100 Switch (New/Workgroup),Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W,NaN
4,4,194,0,Linksys Wireless-G VPN Broadband Silver Router - WRV54G,Linksys Wireless-G VPN Broadband Silver Router - WRV54G/ Built-In VPN Endpoint Capability/ Secur...,149.0,Linksys EtherFast 8-Port 10/100 Switch (New/Workgroup),Linksys EtherFast EZXS88W Ethernet Switch - EZXS88W,NaN
...,...,...,...,...,...,...,...,...,...
160785,160785,1059,1075,Tripp-Lite Black 25 Foot SVGA HD15M To HD15M Computer Monitor Cable - P502025,Tripp-Lite Black 25 Foot SVGA HD15M To HD15M Computer Monitor Cable - P502025/ 25 Foot Length/ F...,NaN,NaN,CASE MATE CARBON FIBER IPHONE 3G CASE BLACK - IPH3GC-BCF,28.08
160786,160786,1065,1075,Logitech V-UBM46 QuickCam Pro 9000 Black Webcam For Windows - 960000048,Logitech V-UBM46 QuickCam Pro 9000 Black Webcam For Windows - 960000048/ Carl Zeiss Optics/ Auto...,NaN,NaN,CASE MATE CARBON FIBER IPHONE 3G CASE BLACK - IPH3GC-BCF,28.08
160787,160787,1068,1075,Case Logic Black In-Car DVD Player Case - PDVK10,Case Logic Black In-Car DVD Player Case - PDVK10/ Holds Up To 10' Portable DVD And Portable iPod...,29.0,NaN,CASE MATE CARBON FIBER IPHONE 3G CASE BLACK - IPH3GC-BCF,28.08
160788,160788,1070,1075,OmniMount 37' - 63' Ultra Low Profile Series Large Flat Panel TV Black Mount - ULPTLBK,OmniMount 37' - 63' Ultra Low Profile Series Large Flat Panel TV Black Mount - ULPTLBK/ Ultra Lo...,NaN,NaN,CASE MATE CARBON FIBER IPHONE 3G CASE BLACK - IPH3GC-BCF,28.08


## Attribute Equivalence Blocker

In [10]:
# Create attribute equivalence blocker on Price 
ab = em.AttrEquivalenceBlocker()
# Block tables using 'year' attribute: same year then include in the canidate set
AE = ab.block_tables(A, B, 'price', 'price', 
                   l_output_attrs=['description', 'name', 'price'],
                   r_output_attrs=['description', 'name', 'price']
                   )

In [11]:
AE

,_id,ltable_id,rtable_id,ltable_description,ltable_name,ltable_price,rtable_description,rtable_name,rtable_price
0,0,1,674,Bose Acoustimass 5 Series III Speaker System - AM53BK/ 2 Dual Cube Speakers With Two 2-1/2' Wide...,Bose Acoustimass 5 Series III Speaker System - AM53BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
1,1,281,674,Tech Craft Dark Cherry Veneto Series TV Stand - SWP60/ 60' Wide Credenza For Flat Panel TV?s And...,Tech Craft Dark Cherry Veneto Series TV Stand - SWP60,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
2,2,337,674,Tech Craft Veneto Series Black TV Stand - ABS60BK/ Supports Up To A 60' Flat Panel TV/ Molded To...,Tech Craft Veneto Series Black TV Stand - ABS60BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
3,3,373,674,Microsoft Office Standard 2007 - 02107746/ Create Documents Faster More Easily And More Intuitiv...,Microsoft Office Standard 2007 - 02107746,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
4,4,472,674,Yamaha 5.1 Channel Home Theater In A Box System In Black - YHT390BK/ New Scene Compressed Music ...,Yamaha 5.1 Channel Home Theater In A Box System In Black - YHT390BK,399.0,Samsung LN22A451 - 22' Widescreen LCD HDTV - 3000:1 Dynamic Contrast Ratio - 8ms Response Time -...,Samsung 4 Series LN22A451 22' LCD TV,399.0
...,...,...,...,...,...,...,...,...,...
337,337,1040,778,Logitech Cordless Desktop MX 5500 Revolution Keyboard And Mouse Combination - 920000383/ Cordles...,Logitech Cordless Desktop MX 5500 Revolution Keyboard And Mouse Combination - 920000383,169.0,10.1 Megapixel - 16:9 - 2x Digital Zoom - 2.7' Active Matrix TFT Color LCD,Sony Cyber-shot DSC-W170 Digital Camera - Red - DSCW170R,169.0
338,338,1072,104,Mitsubishi DLP Black TV Stand - MBS73V/ Matching Base For Mitsubishi WD-73735 And WD-73736 DLP H...,Mitsubishi DLP Black TV Stand - MBS73V,549.0,f/4 to 5.6,Canon EF 70-300mm f/4-5.6 IS USM Telephoto Zoom Lens - 0345B002,549.0
339,339,1072,310,Mitsubishi DLP Black TV Stand - MBS73V/ Matching Base For Mitsubishi WD-73735 And WD-73736 DLP H...,Mitsubishi DLP Black TV Stand - MBS73V,549.0,NaN,PLD4555RFC Professional SpeedClean 24-in. Under Counter Dishwasher (14 Place Settings Stainless ...,549.0
340,340,1072,567,Mitsubishi DLP Black TV Stand - MBS73V/ Matching Base For Mitsubishi WD-73735 And WD-73736 DLP H...,Mitsubishi DLP Black TV Stand - MBS73V,549.0,16:9 - 2.7' Hybrid LCD,Sony Handycam HDR-SR10 High Definition Digital Camcorder - HDRSR10,549.0


## Rule-base Blocker

In [12]:
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()

In [13]:
r = em.get_feature_fn('monge_elkan(alphanumeric(ltuple["name"]), alphanumeric(rtuple["name"]))', block_t, block_s)
em.add_feature(block_f, 'name_name_mel_alphanumeric_alphanumeric', r)

True

In [14]:
block_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x0000010142F87430>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x0000010154828EE0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x0000010154828E50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x0000010154828700>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x0000010154828D30>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x00000101548285E0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x0000010154828AF0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_lev_dist,name,name,None,None,lev_dist,<function name_name_lev_dist at 0x0000010154828940>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,name_name_lev_sim,name,name,None,None,lev_sim,<function name_name_lev_sim at 0x0000010154828550>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,description_description_jac_qgm_3_qgm_3,description,description,qgm_3,qgm_3,jaccard,<function description_description_jac_qgm_3_qgm_3 at 0x00000101548283A0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [15]:
em._block_c['corres']

[('id', 'id'),
 ('name', 'name'),
 ('description', 'description'),
 ('price', 'price')]

In [16]:
rb1 = em.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.4
# rb.add_rule(['name_name_cos_dlm_dc0_dlm_dc0(ltuple, rtuple) < 0.3'], block_f)
rb1.add_rule(['name_name_mel(ltuple, rtuple) < 0.7'], block_f)

'_rule_0'

In [17]:
R1 = rb1.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'], show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:33


In [18]:
R1

,_id,ltable_id,rtable_id,ltable_name,rtable_name
0,0,0,18,Sony Turntable - PSLX350H,Sony VAIO Notebook and AC Adapter Case - VGPAMC2
1,1,0,20,Sony Turntable - PSLX350H,Sony Notebook and AC Adapter Cases - VGPAMC3
2,2,0,23,Sony Turntable - PSLX350H,Sony High Definition Mini DV Cassette - DSL-190-B
3,3,0,32,Sony Turntable - PSLX350H,Sony MiniDV Head Cleaner - DVM12CLD
4,4,0,33,Sony Turntable - PSLX350H,Sony SB-V40S A/V Selector - SBV40S
...,...,...,...,...,...
55011,55011,1075,468,Linksys Wireless-G Range Expander - WRE54G,Linksys RangePlus WRT160N 4-Port Wireless-N Broadband Router - WRT160N
55012,55012,1075,472,Linksys Wireless-G Range Expander - WRE54G,Linksys Wireless-G Internet Video Camera - WVC54GCA
55013,55013,1075,473,Linksys Wireless-G Range Expander - WRE54G,Linksys WGA600N Wireless Gaming Adapter
55014,55014,1075,477,Linksys Wireless-G Range Expander - WRE54G,Linksys WUSB600N Dual-Band Wireless-NUSB Network Adapter - WUSB600N


In [38]:
rb2 = em.RuleBasedBlocker()
rb2.add_rule(['name_name_mel_alphanumeric_alphanumeric(ltuple, rtuple) < 0.7'], block_f)

'_rule_0'

In [ ]:
R2 = rb2.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'], show_progress=True)

0% [##############                ] 100% | ETA: 00:06:48

In [21]:
R2

,_id,ltable_id,rtable_id,ltable_name,rtable_name
0,0,0,23,Sony Turntable - PSLX350H,Sony High Definition Mini DV Cassette - DSL-190-B
1,1,0,53,Sony Turntable - PSLX350H,Sony PS-LX350H Belt-Drive Turntable
2,2,0,55,Sony Turntable - PSLX350H,Sanus Television Turntable - TV/LCDS
3,3,0,58,Sony Turntable - PSLX350H,Sony CDP-CX355 300 Disc MegaStorage CD Changer
4,4,0,62,Sony Turntable - PSLX350H,Sony CDX-565MXRF 10-Disc CD/MP3 Changer
...,...,...,...,...,...
18400,18400,1075,322,Linksys Wireless-G Range Expander - WRE54G,Weber Genesis Series E-310 3841001 60' Freestanding Gas Grill with 637 sq. in. Cooking Surface &...
18401,18401,1075,351,Linksys Wireless-G Range Expander - WRE54G,Nikon 18-200mm 3.5-5.6 G ED DX AFS VR Zoom Wide Angle Nikkor Lens - Niko_215930348
18402,18402,1075,472,Linksys Wireless-G Range Expander - WRE54G,Linksys Wireless-G Internet Video Camera - WVC54GCA
18403,18403,1075,473,Linksys Wireless-G Range Expander - WRE54G,Linksys WGA600N Wireless Gaming Adapter


# Combine Block

In [22]:
G = em.combine_blocker_outputs_via_union([AE, R2, R1, OB])

In [23]:
G

,_id,ltable_id,rtable_id,ltable_description,ltable_name,ltable_price,rtable_description,rtable_name,rtable_price
0,0,0,18,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,VAIO NEOPRENE NOTEBOOK & AC ADAPTER CASE UP TO 17 IN LCD,Sony VAIO Notebook and AC Adapter Case - VGPAMC2,NaN
1,1,0,20,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Clam Shell - Neoprene,Sony Notebook and AC Adapter Cases - VGPAMC3,12.99
2,2,0,23,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Sony DVM-63 HD High-Definition miniDV Videocassette,Sony High Definition Mini DV Cassette - DSL-190-B,NaN
3,3,0,28,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,NaN,SONY 7-11719-70670-0 PS2 8 MB Memory Card 2-pk - 97067,NaN
4,4,0,32,Sony Turntable - PSLX350H/ Belt Drive System/ 33-1/3 and 45 RPM Speeds/ Servo Speed Control/ Sup...,Sony Turntable - PSLX350H,NaN,Head Cleaner,Sony MiniDV Head Cleaner - DVM12CLD,7.95
...,...,...,...,...,...,...,...,...,...
163205,163205,1075,784,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN
163206,163206,1075,785,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN
163207,163207,1075,786,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,LG 2.0 cu.ft. Over the Range Microwave Oven,NaN
163208,163208,1075,798,Linksys Wireless-G Range Expander - WRE54G/ Supports 64/128-Bit WEP Encryption In 802.11b/802.11...,Linksys Wireless-G Range Expander - WRE54G,99.0,NaN,Linksys WRT610N Simultaneous Dual-N Band Wireless Router,NaN


# 3. Matching tuple pairs in the candidate set
In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.

This typically involves the following five steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set
## 3.1 Creating a set of learning-based matchers

First, we randomly sample 450 tuple pairs for labeling purposes.

In [24]:
# Read Perfect Matching
matching_dataset_path = datasets_dir + os.sep + 'abtBuyIdDuplicates.csv'
M = em.read_csv_metadata(matching_dataset_path, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [25]:
len(M)

1076

In [26]:
# Label for data from Matching
G['label'] = 0
for index, row in M.iterrows():
    ltable_id = row['entityId1']
    rtable_id = row['entityId2']
    
    if ((G['ltable_id'] == ltable_id) & (G['rtable_id'] == rtable_id)).any():
        G.loc[(G['ltable_id'] == ltable_id) & (G['rtable_id'] == rtable_id), 'label'] = 1
#     else:
#         print(ltable_id, rtable_id, sep = " --- ")

## 3.2 Splitting the labeled data into development and evaluation set
In this step, we split the labeled data into two sets: development and evaluation. Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [27]:
# Split Matched data
matched = G.loc[G['label'] == 1]

em.set_key(matched,'_id')
em.set_fk_ltable(matched,'ltable_id')
em.set_fk_rtable(matched,'rtable_id')
em.set_ltable(matched, A)
em.set_rtable(matched, B)

matched_train_test = em.split_train_test(matched, train_proportion=0.7)
matched_development =  matched_train_test['train']
matched_evaluation = matched_train_test['test']

In [28]:
# Split Un-Matched data
unmatched = G.loc[G['label'] == 0].head(1000)

em.set_key(unmatched,'_id')
em.set_fk_ltable(unmatched,'ltable_id')
em.set_fk_rtable(unmatched,'rtable_id')
em.set_ltable(unmatched, A)
em.set_rtable(unmatched, B)

# unmatched = em.sample_table(unmatched, 1000)

unmatched_train_test = em.split_train_test(unmatched, train_proportion=0.7)
unmatched_development =  unmatched_train_test['train']
unmatched_evaluation = unmatched_train_test['test']

In [29]:
# Combine 
development = pd.concat([matched_development, unmatched_development])
evaluation = pd.concat([matched_evaluation, unmatched_evaluation])

In [30]:
em.set_key(development,'_id')
em.set_fk_ltable(development,'ltable_id')
em.set_fk_rtable(development,'rtable_id')
em.set_ltable(development, A)
em.set_rtable(development, B)

em.set_key(evaluation,'_id')
em.set_fk_ltable(evaluation,'ltable_id')
em.set_fk_rtable(evaluation,'rtable_id')
em.set_ltable(evaluation, A)
em.set_rtable(evaluation, B)

True

## 3.3 Select the best learning-based matcher
Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Extracting feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation
5. Debugging the matcher (and possibly repeat the above steps)

### 3.3.1 Creating a set of learning-based matchers
First, we need to create a set of learning-based matchers. The following matchers are supported in Magellan: (1) decision tree, (2) random forest, (3) naive bayes, (4) svm, (5) logistic regression, and (6) linear regression.

In [31]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree')
svm = em.SVMMatcher(name='SVM')
rf = em.RFMatcher(name='RF')
nb = em.NBMatcher(name='NB')
lg = em.LogRegMatcher(name='LogReg')
ln = em.LinRegMatcher(name='LinReg')

### 3.3.2 Creating features
Next, we need to create a set of features for the development set. Magellan provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [32]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [33]:
# List the names of the features generated
feature_table['feature_name']

0                                       id_id_exm
1                                       id_id_anm
2                                  id_id_lev_dist
3                                   id_id_lev_sim
4                       name_name_jac_qgm_3_qgm_3
5                   name_name_cos_dlm_dc0_dlm_dc0
6                                   name_name_mel
7                              name_name_lev_dist
8                               name_name_lev_sim
9         description_description_jac_qgm_3_qgm_3
10    description_description_cos_dlm_dc0_dlm_dc0
11                                price_price_exm
12                                price_price_anm
13                           price_price_lev_dist
14                            price_price_lev_sim
Name: feature_name, dtype: object

In [34]:
feature_subset_iter1 = feature_table.drop([0,1,2,3,12,13,14], axis=0)

In [35]:
feature_subset_iter1['feature_name']

4                       name_name_jac_qgm_3_qgm_3
5                   name_name_cos_dlm_dc0_dlm_dc0
6                                   name_name_mel
7                              name_name_lev_dist
8                               name_name_lev_sim
9         description_description_jac_qgm_3_qgm_3
10    description_description_cos_dlm_dc0_dlm_dc0
11                                price_price_exm
Name: feature_name, dtype: object

### 3.3.3 Extracting feature vectors
In this step, we extract feature vectors using the development set and the created features.

In [36]:
# Extract feature vectors
feature_vectors_dev = em.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter1, 
                            attrs_after='label')


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [37]:
# Display first few rows
feature_vectors_dev.head(3)

,_id,ltable_id,rtable_id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,description_description_jac_qgm_3_qgm_3,description_description_cos_dlm_dc0_dlm_dc0,price_price_exm,label
115852,115852,768,613,0.358974,0.400892,0.881441,23.0,0.549020,0.108187,0.226805,NaN,1
142797,142797,938,1013,0.475610,0.666667,0.718041,23.0,0.616667,NaN,NaN,NaN,1
6429,6429,61,77,0.553846,0.668153,0.824079,19.0,0.634615,NaN,NaN,NaN,1


In [203]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_dev))

True

We observe that the extracted feature vectors contain missing values. We have to impute the missing values for the learning-based matchers to fit the model correctly. For the purposes of this guide, we impute the missing value in a column with the mean of the values in that column.

In [204]:
# Impute feature vectors with the mean of the column values.
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\matcher\matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


### 3.3.4 Selecting the best matcher using cross-validation
Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' metric to select the best matcher.

In [205]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=10,
        target_attr='label', 
        metric_to_select_matcher='precision',
        random_state=0)

In [206]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.946330,0.943959,0.943193
1,RF,0.981070,0.963209,0.971727
2,SVM,0.688824,0.496760,0.573933
3,NB,0.970047,0.872401,0.918063
4,LogReg,0.980902,0.908372,0.942764
5,LinReg,0.996800,0.876878,0.932599


In [207]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x00000187CB40F0D0>,10,0.945205,0.971014,0.909091,0.971014,0.971831,0.975000,0.963415,0.918919,0.917808,0.920000,0.946330
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000187C9B42700>,10,1.000000,0.985507,0.987654,1.000000,0.972222,0.975309,0.987654,0.945205,0.957143,1.000000,0.981070
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x00000187CB143310>,10,0.700000,0.644444,0.750000,0.661290,0.645833,0.818182,0.769231,0.600000,0.678571,0.620690,0.688824
3,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x00000187C9B82A60>,10,0.985075,0.966667,0.986842,0.968750,0.941176,0.973684,1.000000,0.940299,0.953846,0.984127,0.970047
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x00000187CB35E970>,10,0.985507,0.969231,1.000000,0.984615,0.944444,1.000000,0.986301,0.954545,0.984375,1.000000,0.980902
5,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x00000187CB35E5E0>,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.983871,0.984127,1.000000,0.996800


### 3.3.5 Debugging matcher
We observe that the best matcher is not getting us to the precision that we expect (i.e > 95%). We debug the matcher to see what might be wrong.

To do this, first we split the feature vectors into train and test.

In [208]:
# # Split feature vectors into train and test
train_test = em.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

In [209]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        target_attr='label')

From the GUI, we observe that using 'name_name_mel' related features result in a lot of false positives. But the monge-elkon similarity measure should have inputs as sets, but the default measure generated from py_entitymatching API is not include the tokenizers itself.  

In [210]:
em.add_feature(feature_table, 'name_name_mel_alphanumeric_alphanumeric', r)

True

In [214]:
feature_table['feature_name']

0                                       id_id_exm
1                                       id_id_anm
2                                  id_id_lev_dist
3                                   id_id_lev_sim
4                       name_name_jac_qgm_3_qgm_3
5                   name_name_cos_dlm_dc0_dlm_dc0
6                                   name_name_mel
7                              name_name_lev_dist
8                               name_name_lev_sim
9         description_description_jac_qgm_3_qgm_3
10    description_description_cos_dlm_dc0_dlm_dc0
11                                price_price_exm
12                                price_price_anm
13                           price_price_lev_dist
14                            price_price_lev_sim
15        name_name_mel_alphanumeric_alphanumeric
Name: feature_name, dtype: object

In [140]:
feature_subset_iter2 = feature_table.drop([0,1,2,3,6,11,12,13,14], axis=0)

In [215]:
feature_subset_iter2['feature_name']

4                       name_name_jac_qgm_3_qgm_3
5                   name_name_cos_dlm_dc0_dlm_dc0
7                              name_name_lev_dist
8                               name_name_lev_sim
9         description_description_jac_qgm_3_qgm_3
10    description_description_cos_dlm_dc0_dlm_dc0
15        name_name_mel_alphanumeric_alphanumeric
Name: feature_name, dtype: object

In [224]:
# Extract feature vectors
feature_vectors_dev = em.extract_feature_vecs(development, 
                            feature_table=feature_subset_iter2, 
                            attrs_after='label')


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [225]:
any(pd.isnull(feature_vectors_dev))

True

In [226]:
# # Impute feature vectors with the mean of the column values.
feature_vectors_dev = em.impute_table(feature_vectors_dev, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\matcher\matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


In [227]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, nb, lg, ln], table=feature_vectors_dev, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        k=10,
        target_attr='label', 
        metric_to_select_matcher='precision',
        random_state=0)

In [145]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.950799,0.957188,0.952408
1,RF,0.981206,0.961497,0.970189
2,SVM,0.712592,0.532467,0.607503
3,NB,0.976457,0.900332,0.936270
4,LogReg,0.982750,0.916099,0.947498
5,LinReg,0.995626,0.873811,0.930275


In [146]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x00000187CDB5F040>,10,0.894737,0.920000,0.961538,0.986111,0.955224,0.965116,0.963855,0.942857,0.972603,0.945946,0.950799
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000187CB259790>,10,1.000000,1.000000,0.986842,1.000000,0.970588,0.988095,0.987805,0.985075,0.972603,0.921053,0.981206
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x00000187CA35EC10>,10,0.632653,0.702128,0.671875,0.701754,0.645161,0.857143,0.816667,0.760870,0.706897,0.630769,0.712592
3,NB,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x00000187CC471E80>,10,1.000000,0.955882,0.972603,0.959459,1.000000,0.987179,0.986667,1.000000,1.000000,0.902778,0.976457
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x00000187CD24E7C0>,10,1.000000,1.000000,0.986486,0.960000,1.000000,0.987500,0.986842,1.000000,1.000000,0.906667,0.982750
5,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x00000187CEEBE880>,10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986111,1.000000,1.000000,0.970149,0.995626


In [147]:
# # Split feature vectors into train and test
train_test = em.split_train_test(feature_vectors_dev, train_proportion=0.5)
train = train_test['train']
test = train_test['test']

In [149]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
        target_attr='label')

## 3.3.6 Black Box Feature

In [106]:
import re

def extract_words(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence)

    words = sentence.split()

    return words

# BLack Box Function
def code_feature(x, y):
    # x, y will be of type pandas series
    
    # get name attribute
    x_name = x['name']
    y_name = y['name']
    # get code in x_name
    x_code = extract_words(x_name)[-1]
    y_extract = extract_words(y_name)
    # check if last names match
    if x_code in y_extract:
        return 1
    else:
        return 0

In [107]:
em.add_blackbox_feature(feature_table, 'blackbox_codefeature_name_name', code_feature)

True

Now, observe the best matcher is achieving the expected precision and we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## 3.4 Evaluating the matching output
Evaluating the matching outputs for the evaluation set typically involves the following four steps:

1. Extracting the feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches
### 3.4.1 Extracting the feature vectors
As before, we extract the feature vectors (using the updated feature table and the evaluation set) and impute it (if necessary).

In [150]:
# Get new set of features
feature_vectors_eval = em.extract_feature_vecs(evaluation, 
                                               feature_table=feature_subset_iter2, 
                                               attrs_after='label')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [151]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.isnull(feature_vectors_eval))

True

In [152]:
# Impute feature vectors
feature_vectors_eval = em.impute_table(feature_vectors_eval, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

C:\Users\DELL\anaconda3\lib\site-packages\py_entitymatching\matcher\matcherutils.py:224: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans


### 3.4.2 Training the matcher
Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [153]:
# Train using feature vectors from the development set
rf.fit(table=feature_vectors_dev, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

### 3.4.3 Predicting the matches
Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [154]:
feature_vectors_eval.shape

(622, 11)

In [155]:
# Predict M 
predictions = rf.predict(table=feature_vectors_eval, 
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], 
                         append=True, 
                         target_attr='predicted', 
                         inplace=False)

### 3.4.4 Evaluating the matching output
Finally, we evaluate the predicted outputs

In [220]:
# Evaluate the result
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 96.53% (306/317)
Recall : 95.03% (306/322)
F1 : 95.77%
False positives : 11 (out of 317 positive predictions)
False negatives : 16 (out of 305 negative predictions)


In [77]:
# Save the model to disk
em.save_object(rf, 'matcher/matcher_abt_buy.pkl')

File already exists at matcher/matcher_abt_buy.pkl; Overwriting it


True